The goal is to interpret the cryptomarket with the help of the 2D map & overlay network obtainded with affinity propagation a better clustering technique.

Clustering is an unsupervised learning tefchnique that aims to group similar objects into clusters while separating from different 
objects.Crypto being filled with real word data, prototype based clustering approaches are not the best approach given that the number of clusters is unknown and difficult to guess which is why we rely on affinity propagation an approach that automatically estimate the number of clusters. 

Affinity propagation identifies clusters by comparing data point with one another by measuring similarities, this approach selects cluster centers that represent other surrounding data points.

***
Make sure to have cmcScraper (pip install cryptocmd), pandas, NumPy, matplotlib & Seaborn installed

In [8]:
# pip install cryptocmd

STEPS:
1/ Defining a list of cryptocurrencies of interest and download their price data from coinmarketcap using cmcscraper
2/ Visualizing the historical price data for better understanding
3/ Preparing and cleaning the data before clustering
4/ Affinity propagation to cluster
5/ Calculating covariances matrixes between clusters
6/ Arranging cryptocurrencies on 2D map to create a network overlay based on covariance
7/ Interpreting the crypto market map

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn import cluster, covariance, manifold
from cryptocmd import CmcScraper

#Define the coins of interest in a dictionary

symbols_dict = {
    'BTC':'Bitcoin',
    'ETH': 'Ethereum',
    'XRP': 'Ripple',
    'ADA': 'Cardano',
    'KMD': 'Komodo',
    'BNB': 'Binance Coin',
    'DOGE': 'Doge Coin',
    'LTC': 'Litecoin',
    'USDT': 'Tether',
    'ZRX': 'Zer0',
    'BAT': 'Battoken',
    'UNI': 'DOT',
    'PSG': 'PSG Token',
    'ACM': 'ACM Token',
    'RSR': 'AS Roma Token',
    'JUV': 'Juventus Token',
    'ATM': 'Atletico Madrid Token',
    'SOL': 'Solana',
    'MATIC': 'Polygon',
    'LINK': 'Link',
    'ETC': 'Ethereum Classic', 
    'AVAX': 'Avalanche', 
    'DCR': 'Decred',
    'WAVES': 'WAVES',
    'VET' : 'Vechain', 
    'ARK' : 'ARK',
    'BCH': 'Bitcoin Cash', 
    'ICP': 'Internet Computer', 
    'DGB': 'DigiByte',
    'BTT': 'Bittorrent',
    'CEL': 'Celsius',
    'SNX': 'Synthetix', 
    'ENJ': 'Enjin',
    'ZIL': 'Ziliqa',
    'CHZ': 'Chiliz', 
    'THETA': 'Theta',
    'XLM': 'Stellar Lumen', 
    'SYS': 'Sys Coin',
    'LRC': 'Loopring',
    'EOS': 'EOS', 
    'NEO': 'NEO',
    'XRM': 'Monero'
}

def load_fresh_data_and_save(symbols_dict, save_path):
    symbols, names = np.array(sorted(symbols_dict.items())).T
    for symbol in symbols:

        #Scraper initialization
        scraper = CmcScraper(symbol)

        df_coin_prices = scraper.get_datagramce()
        df = pd.DataFrame()
        print(f'Fetching prices for {symbol}')

        df[symbol + '_Open'] = df_coin_prices['Open']
        df[symbol + '_Close'] = df_coin_prices['Close']
        df[symbol + '_Avg'] = (df_coin_prices['Close'] + df_coin_prices['Open']) / 2

        df[symbol + '_p'] = (df_coin_prices['Open'] - df_coin_prices['Close']) / df_coin_prices['Open']

        if symbol == symbols[0]:
            df_crypto = df.copy()
        else:
            df_crypto = pd.merge(
                left = df_crypto, right = df , how = "outer",
                left_index = True, right_index = True)

        filter_columns = [s for s in df_crypto.columns if '_p' in s]
        X_df_filtered = df_crypto[filter_columns].copy()
    X_df_filtered.to_csv(save_path + 'historical_crypto_prices.csv')
    return names, symbols, X_df_filtered
save_path = ''